# Housing: coherent assembly of plugins 

### I generated two house's models, and I have invented them.

### I generated the svg files with inkscape and generated the .lines files with cvdlab tool. The full svg files are in directory svg. The name of the .lines files have a structure "nth_model_structure_numberOfFloors.lines", and they are in the main directory.

## Description of the code

### Main function:

In [1]:
def multistorey_house(nFloors, baseString, xfactor, yfactor, zfactor):
	"""
	multistorey_house is a function that return the function that calculate the HPC Model represent the house.
	@param nFloor: represent the number of floors.
	@param baseString: String represent the prefix of the .lines files.
	@param xfactor: Float represent the factor to scale and calculate height.
	@param yfactor: Float represent the factor to scale and calculate height.
	@param zfactor: Float represent the factor to scale and calculate height.
	@return renderWindows: Function that calculate the HPC Model.
	"""
	def renderWindows(XWindow, YWindow, occurrencyWindow):
		"""
		renderWindows is a function that return the function that calculate the HPC Model represent the house.
		@param XWindow: Float list of asix X of the window cells
		@param YWindow: Float list of asix Y of the window cells
		@param occurrencyWindow: Bool matrix that represent the full cell and empty cell.
		@return renderDoors: Function that calculate the HPC Model.
		"""
		def renderDoors(XDoor, YDoor, occurencyDoor):
			"""
			renderDoors is a function that return the function that calculate the HPC Model represent the house.
			@param XDoor: Float list of asix X of the door cells.
			@param YDoor: Float list of asix Y of the door cells.
			@param occurencyDoor: Bool matrix that represent the full cells and empty cells.
			@return renderFloor: Function that calculate the HPC Model.
			"""
			def renderFloor(verts, angle, height):
				"""
				renderFloor is a function that return the HPC Model represent the house.
				@param verts: list of list of integer represent the verts that define the shape of roof bottom.
				@param angle: integer represent the angle used to rotate the planes.
				@param height: integer represent the height of the roof.
				@return house: HPC Model represent the house.
				"""
				all_floor = []
				#building roof model
				with open(verts) as file:
					reader = csv.reader(file, delimiter=",")
					new_verts = []
					for row in reader:
						new_verts.append([float(row[0]), float(row[1])])
					roofModel = roof_builder.buildRoof(new_verts, angle, height)
					roofModel = T([3])([nFloors*3/zfactor])(roofModel)
					roofModel = S([1,2,3])([xfactor*1.09, yfactor*1.09, zfactor])(roofModel)
					roofModel = T([1,2])([-SIZE([1])(roofModel)[0]*0.05, -SIZE([2])(roofModel)[0]*0.05])(roofModel)

				for i in range(nFloors):
					floor_lines = [baseString + '_muri_esterni_'+str(i+1)+'.lines', baseString + '_muri_interni_'+str(i+1)+'.lines', baseString + '_porte_'+str(i+1)+'.lines', baseString + '_finestre_'+str(i+1)+'.lines', baseString + '_scale_'+str(i)+'.lines', baseString + '_scale_'+str(i+1)+'.lines']
					floor = build_floor.ggpl_building_house(floor_lines, 
						windowsDoors.window_main(XWindow,YWindow,occurrencyWindow), 
						windowsDoors.door_main(YDoor, XDoor, occurencyDoor), 
						quarterTurnStairs, i, nFloors-1)
					all_floor.append(floor)
				
				all_floor = STRUCT(all_floor)
				return STRUCT([all_floor, roofModel])
			return renderFloor
		return renderDoors
	return renderWindows

### In this function I builded the roof (workshop_09) and invoke a function ggpl_building_house (workshop_08) for each floor. I use the currying for this main function. 

### List of the variables:
* new_verts: represent the verts in float type.
* roofModel: represent the HPC of the roof.
* floor_lines: List of string represent the name of the files.
* windowsDoors: Imported module represent the workshop_07.
* all_floor: The complete structure without the roof.

### The code of the workshop_09: 

In [2]:
from pyplasm import *
from sympy import *
import numpy as np

def transformList2CoupledList(initList):
	"""
	transformList2CoupledList is a function that return a list containing, for every element in the initList, a couple (python tuple)
	made by the original element and its successor.
	Example : [1,2,3] -> [[1,2], [2,3], [3,1]]
	@param initList: integer list represent the initial list.
	@return coupleList: list of list of integer, represent the couple list.
	"""
	result = []
	for element in range(len(initList)-1):
		result.append([initList[element], initList[element+1]])
	result.append([initList[-1], initList[0]])
	return result

def get4CoefficientsOfThePlane(angle, line):
	"""
	get4CoefficientsOfThePlane is a function that from angle and line, return a list containing the 4 coefficients that describe 
	a plane passing through the line.
	@param angle: integer represent the angle used to rotate the planes.
	@param line: couple represent the verts of the line.
	@return planesParam: list that contain the 4 coefficients. 
	"""
	partialPlane = PROD([POLYLINE(line), QUOTE([2])])
	partialPlane = T([1,2])([-line[0][0], -line[0][1]])(partialPlane)
	partialPlane = ROTN([-angle, [line[1][0] - line[0][0], line[1][1] - line[0][1], 0]])(partialPlane)
	partialPlane = T([1,2])([+line[0][0], +line[0][1]])(partialPlane)
	#obtain 3 points 
	points = []
	points.append(UKPOL(partialPlane)[0][0])
	points.append(UKPOL(partialPlane)[0][1])
	points.append(UKPOL(partialPlane)[0][2])

	x1 = points[0][0]
	x2 = points[1][0]
	x3 = points[2][0]
	y1 = points[0][1]
	y2 = points[1][1]
	y3 = points[2][1]
	z1 = points[0][2]
	z2 = points[1][2]
	z3 = points[2][2]

	#calculate the vectors
	p1 = np.array([x1, y1, z1])
	p2 = np.array([x2, y2, z2])
	p3 = np.array([x3, y3, z3])

	v1 = p3 - p1
	v2 = p2 - p1 
	# this is a vector normal to the plane
	cp = np.cross(v1, v2)
	a, b, c = cp 

	# This evaluates a * x3 + b * y3 + c * z3 which equals d
	d = np.dot(cp, p3)

	return [a,b,c,d]


def buildRoof(verts, angle, height):
	"""
	buildRoof is a function that return a HPC Model represent the roof from the verts, angle and height.
	@param verts: list of list of integer represent the verts that define the shape of roof bottom.
	@param angle: integer represent the angle used to rotate the planes.
	@param height: integer represent the height of the roof.
	@return roof: HPC Model represent the roof.
	"""
	lines = transformList2CoupledList(verts)

	base = SOLIDIFY(POLYLINE(verts + [verts[0]]))

	planes = []

	for line in lines:
		planes.append(get4CoefficientsOfThePlane(angle, line))

	couplePlanes = transformList2CoupledList(planes)

	roofTop = []
	linesEquations = []

	# calculating equations with planes intersection
	for couple in couplePlanes:
		x, y, z = symbols('x y z')
		solved = solve([Eq(couple[0][0]*x+couple[0][1]*y+couple[0][2]*z, couple[0][3]), 
						Eq(couple[1][0]*x+couple[1][1]*y+couple[1][2]*z, couple[1][3])])
		linesEquations.append(solved)
		roofTop.append([round(float(solved[x].subs(z,height)),2), round(float(solved[y].subs(z,height)),2)])

	roofTop.append(roofTop[0])
	terrace = T([3])([height])(SOLIDIFY(POLYLINE(roofTop)))

	coupleLines = transformList2CoupledList(linesEquations)
	roofPitch = []

	#building roof pitches
	for couple in coupleLines:
		base1 = [round(float((couple[0])[x].subs(z,0)),2),round(float((couple[0])[y].subs(z,0)),2),0]
		base2 = [round(float((couple[1])[x].subs(z,0)),2),round(float((couple[1])[y].subs(z,0)),2),0]
		top1 = [round(float((couple[0])[x].subs(z,height)),2),round(float((couple[0])[y].subs(z,height)),2),height]
		top2 = [round(float((couple[1])[x].subs(z,height)),2),round(float((couple[1])[y].subs(z,height)),2),height]
		points = [base1, base2, top2, top1, base1]
		faces = [[1,2,3,4]]
		roofPitch.append(TEXTURE("texture/roof.jpg")(MKPOL([points, faces, 1])))

	roofPitch = STRUCT(roofPitch)

	return STRUCT([TEXTURE("texture/surface.jpg")(terrace), base, roofPitch])



Evaluating fenvs.py..
...fenvs.py imported in 0.008159 seconds


### The code is the same of the workshop_09.

### The code of the workshop_08:

In [3]:
from pyplasm import *
import csv

def build_ladder(fileLadder, xfactor, ladderBuilder):
	"""
	build_ladder is a function that generate a HPC Model represent the ladder of the storey.
	@param fileLadder: String represent the name of the file to read.
	@param xfactor: Float represent a value for calculate the height of the ladder.
	@param ladderBuilder: Module for invoke the function that build a ladder.
	@return ladder: HPC Model represent the ladder.
	"""
	with open(fileLadder, "rb") as file:
		reader = csv.reader(file, delimiter=",")
		ladderX = 0
		ladderY = 0
		minX = 1000000
		minY = 1000000
		for row in reader:
			if(minX > float(row[0])):
				minX = float(row[0])
			elif(minX > float(row[2])):
				minX = float(row[2])
			elif(minY > float(row[1])):
				minY = float(row[1])
			elif(minY > float(row[3])):
				minY = float(row[3])
			if(float(row[0]) == float(row[2])):
				ladderY = float(row[1]) - float(row[3])
			elif(float(row[1]) == float(row[3])):
				ladderX = float(row[0]) - float(row[2])
		if(ladderX < 0):
			ladderX = -ladderX
		elif(ladderY < 0):
			ladderY = -ladderY
		ladder = ladderBuilder.ggpl_single_stair(ladderX, ladderY, 3/xfactor)
		ladder = T([1,2])([minX, minY])(ladder)
		print fileLadder
		return ladder

def build_concrete_objects(fileOpen, offset, xfactor, type, externalWalls, windowsFunction, doorsFunction): 
	"""
	build_concrete_objects is a function that generate a list of HPC Models, that represent the windows or the doors.
	@param fileOpen: String represent the name of the file to read.
	@param offset: Integer represent the offset of the external walls or internal walls.
	@param xfactor: Float represent a value for calculate the height of the window or the door.
	@param type: String that define if the object is a door or window.
	@param externalWalls: HPC Model represent the external walls, for calculate the translation of the asix z.
	@param windowsFunction: Function that generate a HPC Model represent the window.
	@param doorsFunction: Function that generate a HPC Model represent the door.
	@return concreteList: List that contain a HPC Models of windows or doors, based on type.
	"""
	with open(fileOpen, "rb") as file:
		reader = csv.reader(file, delimiter=",")
		concreteList = []
		for row in reader:
			width = 0
			if(float(row[0]) == float(row[2])):
				width = float(row[3]) - float(row[1])
			if(float(row[1]) == float(row[3])):
				width = float(row[2]) - float(row[0])
			#add the cuboid offset.
			width = width + offset 
			if(type == "window"):
				objectConcrete = windowsFunction(width, 12., (SIZE([3])(externalWalls)[0]/2.))
			elif(type == "door"):
				objectConcrete = doorsFunction(width, 12., float(2.5/xfactor))
			if(float(row[0]) == float(row[2])):
				objectConcrete = R([1,2])(-PI/2.)(objectConcrete)
				objectConcrete = T([2])(width)(objectConcrete)
			if(type == "window"):
				objectConcrete = T([1,2,3])([float(row[0]), float(row[1]), (SIZE([3])(externalWalls)[0]/4.)])(objectConcrete)
			elif(type == "door"):
				objectConcrete = T([1,2])([float(row[0]), float(row[1])])(objectConcrete)
			concreteList.append(objectConcrete)
		return concreteList

def create_walls(fileOpen):
	"""
	create_walls is a function that generate a line of the walls.
	@param fileOpen: String represent the name of the file to read.
	@return listWalls: List that contain a line of the internal walls or external walls.
	"""
	with open(fileOpen, "rb") as file:
		reader = csv.reader(file, delimiter=",")
		listWalls = []
		for row in reader:
			listWalls.append(POLYLINE([[float(row[0]), float(row[1])],[float(row[2]), float(row[3])]]))
		return listWalls

def ggpl_building_house(lines, windowsFunction, doorsFunction, ladderBuilder, floorNumber, totalNumberOfFloors):
	"""
	ggpl_building_house is a function that generate the HPC Model represent the n storey structure.
	@param lines: List of string, that contain the name of the files to read.
	@param windowsFunction: Function that generate a HPC Model represent the window.
	@param doorsFunction: Function that generate a HPC Model represent the door.
	@param ladderBuilder: Module for invoke the function that build a ladder.
	@param floorNumber: Integer represent the number of the current floor.
	@param totalNumberOfFloors: Integer represent the number of the total floor.
	@return house_building: HPC Model represent the storey structure.
	"""
	
	#create external walls 
	listExternalWalls = create_walls(lines[0])
	externalWalls = STRUCT(listExternalWalls)

	#create floor
	floor = SOLIDIFY(externalWalls)
	floor = PROD([floor, Q(1)])

	#define scale factor
	xfactor = 15/SIZE([1])(externalWalls)[0]
	yfactor = 15.1/SIZE([2])(externalWalls)[0]

	#remove the space of the ladder from second to the last floor	
	if(floorNumber != 0):
		with open(lines[4], "rb") as file:
			reader = csv.reader(file, delimiter=",")
			minX = 100000
			minY = 100000
			xValue = 0
			yValue = 0
			for row in reader:
				if(minX > float(row[0])):
					minX = float(row[0])
				elif(minX > float(row[2])):
					minX = float(row[2])
				elif(minY > float(row[1])):
					minY = float(row[1])
				elif(minY > float(row[3])):
					minY = float(row[3])
				if(float(row[0]) == float(row[2])):
					yValue = float(row[1]) - float(row[3])
				elif(float(row[1]) == float(row[3])):
					xValue = float(row[0]) - float(row[2])
				if(xValue<0):
					xValue = -xValue
				elif(yValue<0):
					yValue = -yValue
			
			diffCuboid = CUBOID([xValue, yValue, 10])
			diffCuboid = T([1,2])([minX, minY])(diffCuboid)
			floor = DIFFERENCE([floor,diffCuboid])

	#apply offset to external walls
	externalWalls = OFFSET([12,12])(externalWalls)
	externalWalls = PROD([externalWalls, Q(3/xfactor)])

	#create internal walls
	listInternalWalls = create_walls(lines[1])
	internalWalls = STRUCT(listInternalWalls)
	internalWalls = OFFSET([12,12])(internalWalls)
	internalWalls = PROD([internalWalls, Q(3/xfactor)])

	#create a cuboid to remove doors space.
	with open(lines[2], "rb") as file:
		reader = csv.reader(file, delimiter=",")
		doorsList = []
		for row in reader:
			if(float(row[1]) == float(row[3])):
				#remove a bit for asix y
				doorsList.append(POLYLINE([[float(row[0]), float(row[1])-1],[float(row[2]), float(row[3])-1]]))
			else:
				doorsList.append(POLYLINE([[float(row[0]), float(row[1])],[float(row[2]), float(row[3])]]))
	doors = STRUCT(doorsList)
	doors = OFFSET([14,14])(doors)
	doors = PROD([doors, Q(2.5/xfactor)])

	#create a cuboid to remove windows space
	with open(lines[3], "rb") as file:
		reader = csv.reader(file, delimiter=",")
		windowList = []
		for row in reader:
			if(float(row[0]) == float(row[2])):
				#remove a bit for asix X
				windowList.append(POLYLINE([[float(row[0])-1, float(row[1])],[float(row[2])-1, float(row[3])]]))
			else:
				windowList.append(POLYLINE([[float(row[0]), float(row[1])-1],[float(row[2]), float(row[3])-1]]))
	windows = STRUCT(windowList)
	windows = OFFSET([14, 14])(windows)
	windows = PROD([windows, Q(SIZE([3])(externalWalls)[0]/2.)])
	windows = T(3)(SIZE([3])(externalWalls)[0]/4.)(windows)
	frame = STRUCT([externalWalls, internalWalls])
	frame = DIFFERENCE([frame, doors, windows])

	#insert the concrete doors
	doorsConcreteList = build_concrete_objects(lines[2], 14, xfactor, "door", externalWalls, windowsFunction, doorsFunction)
	doorsConcreteList = STRUCT(doorsConcreteList)

	#insert the concrete windows
	windowsConcreteList = build_concrete_objects(lines[3], 14, xfactor, "window", externalWalls, windowsFunction, doorsFunction)
	windowsConcreteList = STRUCT(windowsConcreteList)

	#insert the ladder
	if(floorNumber != totalNumberOfFloors):
		ladder = build_ladder(lines[5], xfactor, ladderBuilder)
		frame = STRUCT([frame, windowsConcreteList, doorsConcreteList, ladder])
	else:
		frame = STRUCT([frame, windowsConcreteList, doorsConcreteList])


	frame = T([3])((3/xfactor)*floorNumber)(frame)
	floor = T([3])((3/xfactor)*floorNumber)(floor)
	frame = (S([1,2,3])([xfactor,yfactor, xfactor])(frame))
	floor = (S([1,2,3])([xfactor,yfactor, xfactor])(floor))
	floor = TEXTURE("texture/parquet.jpg")(floor)
	frame = TEXTURE("texture/wall.jpg")(frame)

	floor = STRUCT([floor, frame])
	return floor


### In this functions I refactored the ggpl_building_house function, and split this in more functions.
### ggpl_building_house proceeds in this step:
* create external walls (with helpers function create_walls)
* create floor 
* remove the space of the ladder (if the storey is more than 1)
* create internal walls (with helpers function create_walls)
* remove the space of the doors (with difference between structure and cuboid generated).
* remove the space of the windows (with difference between structure and cuboid generated).
* insert the doors HPC Models (generated with helpers function build_concrete_objects).
* insert the windows HPC Models (generated with helpers function build_concrete_objects).
* insert the ladder HPC Models (generated with helpers function build_ladder).

### The helpers function build_ladder generate the HPC Model represent the ladder in the correct position (this function invoke the workshop_03), based on parameters in .lines file.
### The helpers function build_concrete_objects generate a list of HPC Model represent the windows of the doors, based on a formal parameter, in the correct position (this function invoke the workshop_07), based on parameters in .lines file.
### The helpers function create_walls, generate the skeleton of the walls (internal or external), without offset.

### The code of the workshop_03:

In [4]:
from pyplasm import *
import math

def generate_steps(stepNumber, tread, riser, stepWidth):
	"""
	generate_steps is a function that, given a stepNumber represent the number of the step, tread represent the depth of the step, 
	riser represent the height of the step and at the end stepWidth that represent the width of the step. 
	Return a list that contain the steps.
	@param stepNumber: Number of the steps
	@param tread: Number, represent depth of the step
	@param riser: Number, represent height of the single step
	@param stepWidth: Number, represent width of the all step
	@return steps: List, that contain all steps.
	"""
	step2d = MKPOL([[[tread, 0],[tread, riser*2], [tread*2, riser*2], [tread*2, riser]], [[1,2,3,4]], None])
	steps = []
	firstStep = CUBOID([tread, riser, stepWidth])
	steps.append(firstStep)

	for i in range(int(stepNumber-1)):
		steps.append(T([1,2])([(tread*i), riser*i])(PROD([step2d, Q(stepWidth)])))

	return steps

def ggpl_single_stair(dx, dy, dz):
	"""
	ggpl_quarter_turn_stairs is a function, that generate a Single Stairs, this function given a dx represent the X value of the box 
	that contain the stairs, dy represent the Y value of the box, and at the end dz that represent the Z value of the box.
	Return a HPC Model.
	@param dx: Number, represent the X value of the box.
	@param dy: Number, represent the Y value of the box.
	@param dz: Number, represent the Z value of the box.
	@return singleStairs: HPC Model of the space frame.
	"""
	riser = dz/20.
	stepNumber = 20

	if(dx > dy):
		firstTread = dx / (stepNumber)
		stepWidth = dy
	else:
		firstTread = dy / (stepNumber)
		stepWidth = dx

	stairs = generate_steps(stepNumber, firstTread, riser, stepWidth)

	singleStairs = MAP([S1,S3,S2])(STRUCT(stairs))
	singleStairs = COLOR(Color4f([193/255., 154/255., 107/255., 1]))(singleStairs)

	return singleStairs


### In the original workshop_03 I generated the quarter turn stairs. However in this workshop this stairs is more difficult, so I modified the code and I generated the single stairs that inserted in the model.

### The code of workshop_07: 

In [5]:
from pyplasm import *

def circle(r):
	def circle0(p):
		alpha = p[0]
		return [r*COS(alpha), r*SIN(alpha)]
	return circle0


def resizeXY(X, Y, occurrency, dx, dz):
	"""
	resizeXY is a function that scale the empty space of a window base on dx and dz.
	@param X: Float list of asix X of the windows cells
	@param Y: Float list of asix Y of the windows cells
	@param occurrency: Bool matrix that represent the full cell and empty cell.
	@param dx: represent a X coordinate of the box
	@param dz: represent a Z coordinate of the box	
	"""
	sumY = sum(Y) 
	sumX = sum(X)
	visitedY = [False]*len(Y)
	for y_index in range(len(Y)):
		update = True
		for x_index in range(len(X)):
			if(occurrency[x_index][y_index] == False):
				update = False 
		if(update):
			sumY = sumY - Y[y_index]
			sumX = sumX - X[y_index]
			dx = dx - X[y_index]
			dz = dz - Y[y_index]

	for x_index in range(len(X)):
		modifyX = False
		for y_index in range(len(Y)):
			if(occurrency[x_index][y_index] == False and visitedY[y_index] == False):
				Y[y_index] = (dz * Y[y_index])/sumY
				visitedY[y_index] = True
				modifyX = True
			if(occurrency[x_index][y_index] == False and visitedY[y_index] == True and not modifyX):
				modifyX = True
		if(modifyX):
			X[x_index] = (dx * X[x_index])/sumX


def window_main(X,Y,occurrency):
	"""
	window_main is a function that generate HPC Model represent the window, parametric by dx, dy and dz.
	@param X: Float list of asix X of the window cells
	@param Y: Float list of asix Y of the window cells
	@param occurency: Bool matrix that represent the full cell and empty cell.
	@return windows_aux: function that return a HPC Model.
	"""
	def window_aux(dx,dy,dz):
		"""
		window_aux is the second level function of window_main.
		@param dx: represent the X value of the box that contain the window
		@param dy: represent the Y value of the box that contain the window
		@param dz: represent the Z value of the box that contain the window
		@return HPC Model: represent the window.
		"""
		resizeXY(X,Y,occurrency, dx, dz)
		result = []
		for x_index in range(len(X)):
			y_quotes = []
			x_sum = sum(X[:x_index])
			for y_index in range(len(Y)):
				if(occurrency[x_index][y_index] == False):
					y_quotes.append(-Y[y_index])
				else:
					y_quotes.append(Y[y_index])
			result.append(PROD([ QUOTE([-x_sum, X[x_index]]), QUOTE(y_quotes)]))
		result.append(BOX([dx,dz,dy]))
		res = STRUCT(result)
		res = PROD([res, Q(dy)])
		res = MAP([S1,S3,S2])(res)
		res = STRUCT([res])
		res = COLOR(Color4f([193/255., 154/255., 107/255., 1]))(res)
		return STRUCT([res])
	return window_aux


def door_main(XDoor,YDoor,occurrency):
	"""
	door_main is a function that generate the HPC Model represent the door, parametric by dx,dy and dz.
	@param XDoor: Float list of asix X of the door cells.
	@param YDoor: Float list of asix Y of the door cells.
	@param occurency: Bool matrix that represent the full cells and empty cells.
	@return door_aux: function that return a HPC Model.
	"""
	def door_aux(dx,dy,dz):
		"""
		door_aux is the second level function of door_main. 
		@param dx: represent the X value of the box that contain the door
		@param dy: represent the Y value of the box that contain the door
		@param dz: represent the Z value of the box that contain the door
		@return HPC Model: represent the door.
		"""
		result = []
		circle_door = MAP(circle(sum(YDoor[4:9])/2.*.6))(INTERVALS(2*PI)(50))
		circle_door = JOIN(circle_door)
		circle_door = PROD([circle_door, Q(dy/4.)])
		circle_door = MAP([S1,S3,S2])(circle_door)
		circle_door = T([1,2,3])([dx/2., dy-dy/4.+0.01, dz/2.])(circle_door)
		circle_door = COLOR(Color4f([93/255., 94/255., 107/255., 1]))(circle_door)
		for x_index in range(len(XDoor)):
			y_quotes = []
			x_sum = sum(XDoor[:x_index])
			for y_index in range(len(YDoor)):
				if(occurrency[x_index][y_index] == False):
					y_quotes.append(-YDoor[y_index])
				else:
					y_quotes.append(YDoor[y_index])
			result.append(PROD([ QUOTE([-x_sum, XDoor[x_index]]), QUOTE(y_quotes)]))
		result.append(BOX([dx,dz,dy]))
		res = STRUCT(result)
		res = PROD([res, Q(dy)])
		res = MAP([S2,S3,S1])(res)
		res = S([1,2,3])([dx/SIZE([1])(res)[0], dy/SIZE([2])(res)[0], dz/SIZE([3])(res)[0]]) (res)
		res = COLOR(Color4f([93/255., 94/255., 107/255., 1]))(res)
		glass = CUBOID([SIZE([1])(res)[0]*0.9, dy/4.*0.9, SIZE([3])(res)[0]*0.9])
		glass = T([1,2,3])([dx*0.05, dy/8. + dy*0.05, dz*0.05])(glass)
		glass = COLOR(Color4f([38/255.,226/255.,189/255.,1]))(glass)
		return STRUCT([res, circle_door, glass])
	return door_aux








### This code is the same of the original workshop_07

## Input and Example:

### I developed two models of the houses, represent by svg files in svg directory, below I describe the .lines file and formal parameter pass for the models.

### .lines files for first model:

#### external walls first floor:

In [7]:
59.258396,406.34328,59.258396,204.01819
59.258396,204.01819,253.118,204.01819
253.118,204.01819,253.118,22.010261000000014
253.118,22.010261000000014,704.32836,22.010261000000014
704.32836,22.010261000000014,704.32836,406.34328
704.32836,406.34328,59.258396,406.34328

(704.32836, 406.34328, 59.258396, 406.34328)

#### internal walls first floor:

In [8]:
252.90858,203.83442,399.86492999999996,203.83442
399.86492999999996,203.83442,399.86492999999996,22.010260999999986
517.24114,22.010261,517.24114,405.92001
704.11672,215.78545,517.24114,215.78545

(704.11672, 215.78545, 517.24114, 215.78545)

#### windows first floor:

In [9]:
298.76544,21.940582,351.98303,21.940582
581.42555,21.940582,644.21297,21.940582
704.31617,294.8558,704.31617,346.03105
200.49962,406.13426,272.85686999999996,406.13426

(200.49962, 406.13426, 272.85686999999996, 406.13426)

#### doors first floor:

In [10]:
114.70707,203.95266,181.00984,203.95266
292.07334,204.01455,351.00135,204.01455
516.65413,79.242859,516.65413,133.5108
516.71249,283.5354,516.71249,336.89733

(516.71249, 283.5354, 516.71249, 336.89733)

#### ladder first floor:

In [11]:
144.8987,319.43619,403.89771999999994,319.43619
403.89771999999994,319.43619,403.89771999999994,405.284762
403.89771999999994,405.284762,144.8987,405.284762
144.8987,405.284762,144.8987,319.43619

(144.8987, 405.284762, 144.8987, 319.43619)

#### for the second floor the external walls and ladder is the same of the first floor. So internal walls second floor:

In [12]:
253.01165,204.05292,704.49219,204.05292
481.25,22.109373,481.25,204.05292
704.06843,291.33641,503.125,291.33641
503.125,291.33641,503.125,406.38671999999997

(503.125, 291.33641, 503.125, 406.38671999999997)

#### windows second floor:

In [13]:
98.539197,203.98386,163.51844,203.98386
340.91797,21.91406,385.15625,21.91406
583.3631,21.959109,636.71875,21.959109
704.34465,332.07791,704.34465,367.57135999999997
59.316868,278.35437,59.316868,333.4375

(59.316868, 278.35437, 59.316868, 333.4375)

#### doors second floor:

In [14]:
330.21334,203.91481,395.3125,203.91481
575.97657,203.94531,626.4524200000001,203.94531
503.125,332.85156,503.125,370.88592

(503.125, 332.85156, 503.125, 370.88592)

#### I invoke the first model with:

In [16]:
from pyplasm import *
import csv
import src.workshop_08 as build_floor
import src.workshop_07 as windowsDoors
import src.workshop_03 as quarterTurnStairs
import src.workshop_09 as roof_builder

XWindow = [2,3,2,3,2]
YWindow = [2,3,2,3,2]
occurrencyWindow = [[True, True, True, True, True],
			  [True, False, True, False, True],
			  [True, True, True, True, True],
			  [True, False, True, False, True],
			  [True, True, True, True, True]]

XDoor = [.2, .2, .05, .2, .05, .2, .3, .2, .05, .2 ,.05, .2, .2]
YDoor = [.2, .2, .05, .2, .05, 1, .05, .2, .05, .2, .2]
occurencyDoor = [[True, True, True, True, True, True, True, True, True, True, True, True, True],
				[True, False, False, False, False, False, True, False, False, False, False, False, True],
				[True, False, True, True, True, True, True, True, True, True, True, False, True], 
				[True, False, True, False, False, False, True, False, False, False, True, False, True],
				[True, False, True, False, True, True, True, True, True, False, True, False, True],
				[True, False, True, False, True, False, True, False, True, False, True, False, True], 
				[True, False, True, False, True, True, True, True, True, False, True, False, True],
				[True, False, True, False, False, False, True, False, False, False, True, False, True],
				[True, False, True, True, True, True, True, True, True, True, True, False, True],
				[True, False, False, False, False, False, True, False, False, False, False, False, True],
				[True, True, True, True, True, True, True, True, True, True, True, True, True]]

listExternalWalls = build_floor.create_walls('first_model_muri_esterni_1.lines')
externalWalls = STRUCT(listExternalWalls)
xfactor = 15/SIZE([1])(externalWalls)[0]
yfactor = 15.1/SIZE([2])(externalWalls)[0]
zfactor = xfactor

multistorey_house(2, 'first_model', xfactor, yfactor, zfactor)(XWindow, YWindow, occurrencyWindow)(XDoor, YDoor, occurencyDoor)('first_model_muri_esterni_1.lines', PI/5., 3/zfactor)

first_model_scale_1.lines


<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x112d31ae0> >

### This is the example image of the first model:

![title](img/Example_1.png)

![title](img/Example_3.png)

### .lines files for the second model:

#### external walls first floor:

In [17]:
65.186407,51.583016,700.47766,51.583016
700.47766,51.583016,700.47766,404.03155
700.47766,404.03155,65.18640700000003,404.03155
65.18640700000003,404.03155,65.186407,51.583016

(65.18640700000003, 404.03155, 65.186407, 51.583016)

#### internal walls first floor:

In [18]:
564.0625,51.406248,564.0625,283.4375
564.0625,283.4375,700.33955,283.4375
564.02815,283.46432,444.15144999999995,283.46432
444.15144999999995,283.46432,444.15144999999995,404.41683
444.14063,283.4375,351.34368,283.4375
351.34368,283.4375,351.34368,404.06608

(351.34368, 283.4375, 351.34368, 404.06608)

#### windows first floor:

In [19]:
65.136719,177.38281,65.136719,200.73835
169.11177,51.583016,208.81747000000001,51.583016
388.49441,51.444909,428.13106000000005,51.444909
700.48829,122.30469,700.48829,151.40625
700.47766,329.86821,700.47766,360.39061999999996
549.51172,404.04297,583.98438,404.04297
383.10825,403.89345,408.79611,403.89345

(383.10825, 403.89345, 408.79611, 403.89345)

#### doors first floor:

In [20]:
65.186407,80.033015,65.186407,113.73107
564.0625,123.28125,564.0625,173.28125
483.09757,283.32622,531.43494,283.32622
378.1364,283.39527,406.25,283.39527

(378.1364, 283.39527, 406.25, 283.39527)

#### ladder first floor:

In [21]:
132.12988,312.75574,312.9398,312.75574
312.9398,312.75574,312.9398,401.862758
312.9398,401.862758,132.12988,401.862758
132.12988,401.862758,132.12988,312.75574

(132.12988, 401.862758, 132.12988, 312.75574)

#### for the second floor the external walls and ladder is the same of the first floor. So internal walls second floor:

In [22]:
65.185547,138.61328,700.5467100000001,138.61328
265.64842,51.583016,265.64842,138.56445
495.32002,51.513962,495.32002,138.71094
495.32002,138.71094,495.32002,404.10061

(495.32002, 138.71094, 495.32002, 404.10061)

#### windows second floor:

In [23]:
65.039063,169.17969,65.039063,202.94806
65.234375,260.87891,65.234375,305.69952
409.90096,404.03155,451.88543,404.03155
567.38282,403.94531,646.875,403.94531
700.33955,214.82525,700.33955,251.40625
578.46031,51.513962,621.875,51.513962
331.9336,51.60156,359.76563,51.60156
126.95313,51.503904,151.5625,51.503904

(126.95313, 51.503904, 151.5625, 51.503904)

#### doors second floor:

In [24]:
141.79688,138.56445,190.625,138.56445
367.9165,138.5903,423.4375,138.5903
495.38907,77.270879,495.38907,104.53125
570.89844,138.61328,620.9281400000001,138.61328
495.26368,243.10547,495.26368,302.38495

(495.26368, 243.10547, 495.26368, 302.38495)

#### I invoke the first model with:

In [25]:
from pyplasm import *
import csv
import src.workshop_08 as build_floor
import src.workshop_07 as windowsDoors
import src.workshop_03 as quarterTurnStairs
import src.workshop_09 as roof_builder

XWindow = [2,3,2,3,2]
YWindow = [2,3,2,3,2]
occurrencyWindow = [[True, True, True, True, True],
			  [True, False, True, False, True],
			  [True, True, True, True, True],
			  [True, False, True, False, True],
			  [True, True, True, True, True]]

XDoor = [.2, .2, .05, .2, .05, .2, .3, .2, .05, .2 ,.05, .2, .2]
YDoor = [.2, .2, .05, .2, .05, 1, .05, .2, .05, .2, .2]
occurencyDoor = [[True, True, True, True, True, True, True, True, True, True, True, True, True],
				[True, False, False, False, False, False, True, False, False, False, False, False, True],
				[True, False, True, True, True, True, True, True, True, True, True, False, True], 
				[True, False, True, False, False, False, True, False, False, False, True, False, True],
				[True, False, True, False, True, True, True, True, True, False, True, False, True],
				[True, False, True, False, True, False, True, False, True, False, True, False, True], 
				[True, False, True, False, True, True, True, True, True, False, True, False, True],
				[True, False, True, False, False, False, True, False, False, False, True, False, True],
				[True, False, True, True, True, True, True, True, True, True, True, False, True],
				[True, False, False, False, False, False, True, False, False, False, False, False, True],
				[True, True, True, True, True, True, True, True, True, True, True, True, True]]

listExternalWalls2 = build_floor.create_walls('second_model_muri_esterni_1.lines')
externalWalls2 = STRUCT(listExternalWalls2)
xfactor2 = 15/SIZE([1])(externalWalls2)[0]
yfactor2 = 15.1/SIZE([2])(externalWalls2)[0]
zfactor2 = xfactor2

multistorey_house(2, 'second_model', xfactor2, yfactor2, zfactor2)(XWindow, YWindow, occurrencyWindow)(XDoor, YDoor, occurencyDoor)('second_model_muri_esterni_1.lines', PI/5., 3/zfactor2)

second_model_scale_1.lines


<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x113939f00> >

### This is the example image of the second model:

![title](img/Example_2.png)